# Setup

In [1]:
import os
import json

with open(".env.json", "r") as f_in:
    env_json: dict = json.load(f_in)
    for key, value in env_json.items():
        os.environ[key] = value

# Helper libs

In [ ]:
from Embedder import Embedder
def print_length_cost(content, chain_numbers=3, price_per_1k=0.03):
    
    print("Expected chars: ", len(content))
    print("Expected words: ", len(content.split()))
    input_tokens = Embedder.num_tokens_from_string(content)
    print("Expected tokens: ", input_tokens)

    expected_cost_input = Embedder.get_embedding_cost(input_tokens, price_per_1k=price_per_1k)
    print("Expected total cost: $", expected_cost_input * chain_numbers)

In [2]:
import sys
import types
def reload_package(root_module):
    package_name = root_module.__name__

    # get a reference to each loaded module
    loaded_package_modules = dict([
        (key, value) for key, value in sys.modules.items() 
        if key.startswith(package_name) and isinstance(value, types.ModuleType)])

    # delete references to these loaded modules from sys.modules
    for key in loaded_package_modules:
        del sys.modules[key]

    # load each of the modules again; 
    # make old modules share state with new modules
    for key in loaded_package_modules:
        print('loading %s' % key)
        newmodule = __import__(key)
        oldmodule = loaded_package_modules[key]
        oldmodule.__dict__.clear()
        oldmodule.__dict__.update(newmodule.__dict__)

# PoC Simple Calculator

In [2]:
file_content = ""
with open("./input/project_01/sample.cs", "r") as f_in:
    file_content = f_in.read()

In [3]:
# Instancia de modelo a reusarse
from framework_genai.GenAIMemory import GenAIMemory, EnumMemoryType
from framework_genai.GenAILLM import GenAILLM, EnumGenAIPlatforms, EnumGenAIModelsIdsOpenAI
from framework_genai.LoggingAndTelemetry import EnumLogs
import os

ai_prefix = "Human"
human_prefix = "Assistant"

gen_ai_llm = GenAILLM(
    platform = EnumGenAIPlatforms.PLATFORM_OPENAI,
    model_id = EnumGenAIModelsIdsOpenAI.MODEL_CHAT_GPT_3_5_TURBO,
    parameters_inference = {
        'max_tokens': 1024, 
        "temperature": 0,
        "top_p": 0.2
    },
    ai_prefix = ai_prefix,
    human_prefix = human_prefix,
    component_memory=GenAIMemory(EnumMemoryType.MEMORY_CHAT_BUFFER, ai_prefix, human_prefix),
    platform_configuration = {
        "OPENAI_API_KEY": os.environ["OPENAI_API_KEY"]
    },
    verify_ssl=True,
    verbose_level=EnumLogs.LOG_LEVEL_DEBUG
)

2024-02-08 08:38:51.012058-05:00: Modelo EnumGenAIModelsIdsOpenAI.MODEL_CHAT_GPT_3_5_TURBO construido


In [4]:
from framework_genai.GenAIOutputParser import GenAIOutputParser
gen_ai_output_parser = GenAIOutputParser([
    {
        "name": "reasoning",
        "description": "razonamiento detras de la respuesta"
    },
    {
        "name": "respuesta",
        "description": "respuesta final concreta y concisa"
    },
])

In [5]:
from framework_genai.GenAIPrompt import GenAIPrompt

PROMPT_TRANSCODER =  """{system_prompt}. Tienes la capacidad de realizar lo siguiente:
* Migrar código desde un lenguaje hacia otro conservando la funcionalidad, es decir, debe poder ejecutarse y 
dar los mismos resultados en el nuevo lenguaje. Migras el código ubicado en la etiqueta <code-origin> desde un lenguaje de programación
que debes entender cuál es hacia el lenguaje de programación <language-objetive>
* Generas documentación de código de cada método, función, etc. En el código migrado
* Generar pruebas unitarias listas para ser ejecutadas para las funciones que puedan testearse, incluyendo ejemplos de entrada y salida esperada.
* Formatear la respuesta que das hacia un formato json según las instrucciones en <instrucciones>
* Apoyarte en la etiqueta <contexto> para mejorar el entendimiento y resultado de tu respuesta final

<instrucciones>
{format_instructions}
</instrucciones>

<contexto>
La estructura del proyecto es la siguiente:
project_01
|-sample.cs
La estructura del proyecto que contendrá la migración es el siguiente:
project_01
|-sample.py
|-sample_unit_tests.py

sample_unit_tests.py será el archivo que contendrá las pruebas unitarias generadas. Recuerda hacer una importación de lo que se requiera de sample.py para
que el archivo de tests pueda ejecutarse sin error.
El código estará en el archivo {filename}
</contexto>

<code-origin>
{code}
</code-origin>

<language-objetive>
{programming_language_objective}
</language-objetive>
"""

gen_ai_output_parser_transcoder = GenAIOutputParser(response_schemas=[
    {'name': 'codigo_migrado', 'description': 'Es el código de programación resultado luego de migrarlo. Además cada clase y método contiene documentación'},
    {'name': 'pruebas_unitarias', 'description': 'es el código que contiene la llamada a las funciones para hacer las pruebas unitarias'}
])

chain_transcoder = gen_ai_llm.create_chain(
    gen_ai_prompt=GenAIPrompt(
        PROMPT_TRANSCODER,
        partials={
            "system_prompt": "Eres un experto programador que maneja múltiples lenguajes de programación, resuelves las tareas que te pidan en <peticion>. ",
            "format_instructions": gen_ai_output_parser_transcoder.get_instructions()
        }
    ),
    output_key="output"
)

result_chain_01 = chain_transcoder.invoke({
    "programming_language_objective": "python",
    "code": file_content,
    "filename": "sample.cs"
})

output_01 = gen_ai_output_parser_transcoder.parse(result_chain_01["output"])
print(output_01)



> Entering new LLMChain chain...
Prompt after formatting:
Eres un experto programador que maneja múltiples lenguajes de programación, resuelves las tareas que te pidan en <peticion>. . Tienes la capacidad de realizar lo siguiente:
* Migrar código desde un lenguaje hacia otro conservando la funcionalidad, es decir, debe poder ejecutarse y 
dar los mismos resultados en el nuevo lenguaje. Migras el código ubicado en la etiqueta <code-origin> desde un lenguaje de programación
que debes entender cuál es hacia el lenguaje de programación <language-objetive>
* Generas documentación de código de cada método, función, etc. En el código migrado
* Generar pruebas unitarias listas para ser ejecutadas para las funciones que puedan testearse, incluyendo ejemplos de entrada y salida esperada.
* Formatear la respuesta que das hacia un formato json según las instrucciones en <instrucciones>
* Apoyarte en la etiqueta <contexto> para mejorar el entendimiento y resultado de tu respuesta final

<instrucc

In [10]:
gen_ai_output_parser_documenter = GenAIOutputParser(response_schemas=[
    {'name': 'codigo_migrado_documentado', 'description': 'Código documentado (métodos, clases, etc)'}
])

chain_documenter = gen_ai_llm.create_chain(
    gen_ai_prompt=GenAIPrompt(
        """Tu tarea es agregar documentación al siguiente código hecho en {programming_language_objective} en toda clase, función, etc que falte.
        La documentación debe contener qué hace la función, los argumentos que recibe (si es posible tipadas) y las salidas (si es posible tipadas).
        Agrega la documentación luego de la definición de la clase también, esta definición puede ser incluso un resumen de lo que hacen el resto de métodos de la clase
        El código es:
        {code}
        Tu formato de salida es: {format_instructions}""",
        partials={
            "system_prompt": "",
            "format_instructions": gen_ai_output_parser_documenter.get_instructions()
        }
    ),
    output_key="output"
)

result_chain_02 = chain_documenter.invoke({
    "programming_language_objective": "python",
    "code": output_01["codigo_migrado"]
})

output_02 = gen_ai_output_parser_documenter.parse(result_chain_02["output"])

print(output_02)



> Entering new LLMChain chain...
Prompt after formatting:
Tu tarea es agregar documentación al siguiente código hecho en python en toda clase, función, etc que falte.
        La documentación debe contener qué hace la función, los argumentos que recibe (si es posible tipadas) y las salidas (si es posible tipadas).
        Agrega la documentación luego de la definición de la clase también, esta definición puede ser incluso un resumen de lo que hacen el resto de métodos de la clase
        El código es:
        class Calculator:
    def Add(self, a, b):
        return a + b

    def Subtract(self, a, b):
        return a - b

    def Multiply(self, a, b):
        return a * b

    def Divide(self, a, b):
        if b == 0:
            raise ZeroDivisionError('Cannot divide by zero.')
        return float(a) / b


calculator = Calculator()

print('10 + 5 =', calculator.Add(10, 5))
print('10 - 5 =', calculator.Subtract(10, 5))
print('10 * 5 =', calculator.Multiply(10, 5))
print('10 / 5 =

In [11]:
with open("./output/project_01/sample.py", "w", encoding='utf-8') as f_out:
    f_out.write(output_02['codigo_migrado_documentado'])

with open("./output/project_01/sample_unit_tests.py", "w", encoding='utf-8') as f_out:
    f_out.write(output_01['pruebas_unitarias'])

# Extract SQL Insights

In [ ]:
from Embedder import Embedder

In [8]:
os.listdir("./temp/oracle_query")

['part_main.txt', 'part_rcase.txt', 'part_table_rf.txt', 'part_table_rv.txt']

In [76]:
import os
import re

file_content_sql = {}

base_path = "./temp/oracle_query"

files = os.listdir(base_path)

for file in files:
    filename = file.split(".")[0]

    with open(f"{base_path}/{file}", "r", encoding="utf-8") as f_in:
        file_content_sql[filename] = re.sub('\s+', ' ', f_in.read().replace("\n", "#NEWLINE#")).replace("#NEWLINE#", "\n")  # re.sub('\s+', ' ', f_in.read())

In [77]:
filename = "part_table_rv_01_02"
file_content_01 = file_content_sql[filename]
print_length_cost(file_content_01)
print(file_content_01[:100])

Expected chars:  4788
Expected words:  474
Expected tokens:  1714
Expected total cost: $ 0.15426
 SELECT 
 --ESTADO_TITULO
 CASE
 WHEN Trim(tp.m_cmp_typo) LIKE 'BCE Fund%' THEN 'F'
 ELSE 'L'
 END A


In [81]:
# Instancia de modelo a reusarse
from framework_genai.GenAIMemory import GenAIMemory, EnumMemoryType
from framework_genai.GenAILLM import GenAILLM, EnumGenAIPlatforms, EnumGenAIModelsIdsOpenAI
from framework_genai.LoggingAndTelemetry import EnumLogs
import os

ai_prefix = "Human"
human_prefix = "Assistant"

gen_ai_llm = GenAILLM(
    platform = EnumGenAIPlatforms.PLATFORM_OPENAI,
    model_id = EnumGenAIModelsIdsOpenAI.MODEL_CHAT_GPT_3_5_TURBO,
    parameters_inference = {
        'max_tokens': 2048, 
        "temperature": 0.1,
        "top_p": 0.3
    },
    ai_prefix = ai_prefix,
    human_prefix = human_prefix,
    component_memory=GenAIMemory(EnumMemoryType.MEMORY_CHAT_BUFFER, ai_prefix, human_prefix),
    platform_configuration = {
        "OPENAI_API_KEY": os.environ["OPENAI_API_KEY"]
    },
    verify_ssl=True,
    verbose_level=EnumLogs.LOG_LEVEL_DEBUG
)

c:\Users\magod\anaconda3\envs\env_ai\Lib\site-packages\langchain_openai\chat_models\base.py:299: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  warnings.warn(


2024-02-08 14:52:16.546724-05:00: Modelo EnumGenAIModelsIdsOpenAI.MODEL_CHAT_GPT_3_5_TURBO construido


In [82]:
from framework_genai.GenAIPrompt import GenAIPrompt

In [83]:
PROMPT_CLEANER =  """{system_prompt}
Recibirás un código SQL en Oracle en la etiqueta <sql-oracle> y tu tarea es limpiar ese código. Específicamente eliminando comentarios.
Por ejemplo si recibes lo siguiente de entrada:
'''
----------Renta Variable-----------
SELECT
    --INSTITUCION_GRUPO
    rv.inst_grupo,
    --FECHA_REPORTE
    To_char(rv.fec_rep,'YYYY-MM-DD') fec_rep,
    --TIPO_INSTRUMENT
    rv.tipo_inst AS tipo_inst
    --CODIGO_SUCURSAL
FROM rv
'''
Tu salida debería ser:
'''
SELECT
    rv.inst_grupo,
    To_char(rv.fec_rep,'YYYY-MM-DD') fec_rep,
    rv.tipo_inst AS tipo_inst
FROM rv
'''

<sql-oracle>
{code}
</sql-oracle>
"""

chain_sql_clean = gen_ai_llm.create_chain(
    gen_ai_prompt=GenAIPrompt(
        PROMPT_CLEANER,
        partials={
            "system_prompt": "Eres un experto programador que maneja Oracle SQL.",
            "code": file_content_01
        }
    ),
    output_key="output"
)

result_chain_01 = chain_sql_clean.invoke({
    "contexto": "",
    "code": file_content_01
})

output_clean = result_chain_01["output"]

with open("./output/output_test_clean.txt", "w") as f_out:
    f_out.write(output_clean)



> Entering new LLMChain chain...
Prompt after formatting:
Eres un experto programador que maneja Oracle SQL.
Recibirás un código SQL en Oracle en la etiqueta <sql-oracle> y tu tarea es limpiar ese código. Específicamente eliminando comentarios.
Por ejemplo si recibes lo siguiente de entrada:
'''
----------Renta Variable-----------
SELECT
    --INSTITUCION_GRUPO
    rv.inst_grupo,
    --FECHA_REPORTE
    To_char(rv.fec_rep,'YYYY-MM-DD') fec_rep,
    --TIPO_INSTRUMENT
    rv.tipo_inst AS tipo_inst
    --CODIGO_SUCURSAL
FROM rv
'''
Tu salida debería ser:
'''
SELECT
    rv.inst_grupo,
    To_char(rv.fec_rep,'YYYY-MM-DD') fec_rep,
    rv.tipo_inst AS tipo_inst
FROM rv
'''

<sql-oracle>
 SELECT 
 --ESTADO_TITULO
 CASE
 WHEN Trim(tp.m_cmp_typo) LIKE 'BCE Fund%' THEN 'F'
 ELSE 'L'
 END AS estitulo,
 --TIPO_RENTA
 'V' AS tiporenta,
 --TIPO_ID_EMISOR
 Nvl(
 (
 SELECT m_iden_tipo
 FROM parties_rep
 WHERE Trim(m_dsp_label) LIKE Trim(tp.m_tp_seciss)),' ') AS tipoidemi,
 --ID_EMISOR
 Nvl(
 (
 SELE

In [84]:
PROMPT_TRANSCODER_ORACLE =  """{system_prompt}
Recibirás un text con una consulta muy grande en SQL Oracle en la etiqueta <sql-oracle>, tu tarea es obtener la trazabilidad de cada columna en el formato de una tabla con las siguientes columnas:
1. columna_final .- Es simplemente el nombre de la columna resultante de procesar todo el SQL
2. tablas_que_la_generan .- Corresponde a las tablas originales o la tabla original que genera a la columna. A veces para generar el campo se involucran 2 o más tablas unidas por un JOIN
3. calculos .- Cálculos que se utilizan con las tablas originales o la tabla original
4. descripcion .- Descripción de qué podría significar esta columna

A continuación te daré un ejemplo si la entrada es:
'''
{sample_code}
'''
Luego de analizar el código anterior de ejemplo tu salida podría ser:
'''
columna_final|tablas_que_la_generan|calculos|descripcion
INST_GRUPO|param_rep|(SELECT valor FROM param_rep WHERE codigo=20)|Grupo de la institución
FEC_REP|dyn_audit_rep|(SELECT m_rep_date2 FROM dyn_audit_rep WHERE  trim(m_outputtbl) = 'DM_NETTING_MCALL.REP' AND m_ref_data = nmc.m_ref_data AND rownum = 1)|Fecha del reporte
TIPCUP|rf|CASE WHEN ( ( rf.valint=0) AND ( rf.val_nom<>0)) THEN '1' WHEN ( ( rf.valint<>0) AND ( rf.val_nom=0)) THEN '2' WHEN ( ( rf.valint<>0) AND ( rf.val_nom<>0)) THEN '3' ELSE '0' END;CASE WHEN ((RV.VALINT=0) and (RV.VAL_NOM<>0)) THEN '1' WHEN ((RV.VALINT<>0) and (RV.VAL_NOM=0)) THEN '2' WHEN ((RV.VALINT<>0) and (RV.VAL_NOM<>0)) THEN '3' ELSE '0' END|Tipo de cupón
industry|tp_all_rep,securities_rep|SELECT ( SELECT nvl(upper(m_descval),' ') FROM secutypo_val_rep WHERE trim(m_lbgr) LIKE 'INDUSTRY' AND trim(sec.m_se_d_label)=trim(tp.m_instrument) ) AS industry FROM tp_all_rep TP INNER JOIN mx_trn_udf_rep ETU ON etu.m_nb=tp.m_udf_ref2 AND etu.m_ref_data=tp.m_ref_data INNER JOIN parties_rep PR ON Trim(tp.m_tp_cntrp) LIKE Trim(pr.m_dsp_label) INNER JOIN dm_securities_md_rep SMD ON Trim(smd.m_label) LIKE Trim(tp.m_instrument) AND tp.m_ref_data=smd.m_ref_data INNER JOIN securities_rep SEC ON Trim(sec.m_se_d_label) LIKE Trim(tp.m_instrument) AND tp.m_ref_data=sec.m_ref_data INNER JOIN portfolio_rep PL ON Trim(pl.m_dsp_label)= Trim(tp.m_tp_pfolio) INNER JOIN currency_rep CR ON Trim(cr.m_symbol)=Trim(smd.m_currency) WHERE tp.m_ref_data = @MxDataSetKey:N AND trim(tp.m_cmp_typo) IN ('ETF', 'MM Fund', 'BCE Fund') AND trim(tp.m_stp_status) LIKE 'Matched%' AND trim(tp.m_tp_status2) NOT LIKE 'DEAD%' AND trim(tp.m_stp_status) NOT LIKE 'Cancelled%' AND ( tp.m_tp_ipaydte<=tp.m_rep_date2) AND trim(tp.m_amd_sts2) NOT LIKE 'CA' AND trim(m_tp_buy) NOT LIKE 'S' )|Industria
'''
Es importante que notes que en este ejemplo mis tablas origen no son 'RV' en el ejemplo, ya que no son la fuente más profunda de donde viene la información.
TIPCUP tiene 2 calculos involucrados el que se hizo en la tabla origen y el que se hizo para la tabla final
En el caso de 'industry' hay 2 o más tablas involucradas, fíjate que no usé su alias 'tp' o 'sec' sino su nombre verdadero; en este caso no se puede determinar del todo cómo se generó al 100%. Además en el cálculo puse el SQL entero debido a la abiguedad de a dónde pertenecen algunos campos

Las personas de negocio proporcionaron un contexto adicional que posiblemente sirva de ayuda para que entiendas lo que hay que hacer en la etiqueta <contexto>

<contexto>{contexto}</contexto>

<sql-oracle>
{code}
</sql-oracle>

Para tu salida solo utiliza una representación de tabla como antes te mostré
"""

sample_code = ""
with open("./temp/sample_code.txt", "r", encoding="utf-8") as f_in:
    sample_code = f_in.read()

chain_sql_insighter = gen_ai_llm.create_chain(
    gen_ai_prompt=GenAIPrompt(
        PROMPT_TRANSCODER_ORACLE,
        partials={
            "system_prompt": "Eres un experto programador que maneja pyspark y Oracle SQL.",
            "sample_code": sample_code
        }
    ),
    output_key="output"
)

result_chain_sql_insighter = chain_sql_insighter.invoke({
    "contexto": "Existen algunos comentarios en el SQL que podrían sugerir la descripción de las columnas finales",
    "code": output_clean
})

output_chain_sql_insighter = result_chain_sql_insighter["output"]

with open(f"./output/{filename}.txt", "w", encoding="utf-8") as f_out:
    f_out.write(output_chain_sql_insighter)



> Entering new LLMChain chain...
Prompt after formatting:
Eres un experto programador que maneja pyspark y Oracle SQL.
Recibirás un text con una consulta muy grande en SQL Oracle en la etiqueta <sql-oracle>, tu tarea es obtener la trazabilidad de cada columna en el formato de una tabla con las siguientes columnas:
1. columna_final .- Es simplemente el nombre de la columna resultante de procesar todo el SQL
2. tablas_que_la_generan .- Corresponde a las tablas originales o la tabla original que genera a la columna. A veces para generar el campo se involucran 2 o más tablas unidas por un JOIN
3. calculos .- Cálculos que se utilizan con las tablas originales o la tabla original
4. descripcion .- Descripción de qué podría significar esta columna

A continuación te daré un ejemplo si la entrada es:
'''
----------Renta Variable-----------
select
        --INSTITUCION_GRUPO
        RV.INST_GRUPO,
        --FECHA_REPORTE
        TO_CHAR(RV.FEC_REP,'YYYY-MM-DD') FEC_REP,   
        --TIPO_CUPO

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens. However, you requested 5214 tokens (3166 in the messages, 2048 in the completion). Please reduce the length of the messages or completion.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

# END